In [1]:
import os
import sys
import json

curdir = os.getcwd()
while 'config.json' not in os.listdir(curdir):
    curdir = os.path.dirname(curdir)
with open(os.path.join(curdir,'config.json'), 'r') as f:
    dataset = json.load(f)['dataset']

curdir = os.getcwd()
while 'filepathhelper.py' not in os.listdir(curdir):
       curdir = os.path.dirname(curdir)
sys.path.append(curdir)
import filepathhelper

In [2]:
from neo4j import GraphDatabase
import pandas as pd
import numpy as np
# from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm

# Initialize DB and read csv

In [3]:
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "jrdb"))

In [4]:
def create_person(tx,username):
    tx.run("CREATE (:Person {name: $username})",username=username)
def create_work_relation(tx,userA,userB):
    tx.run("MATCH (a:Person),(b:Person)"
           "WHERE a.name = $userA AND b.name = $userB AND NOT EXISTS((a)-[:WorkWith]->(b))"
           "CREATE (a)-[:WorkWith {weight: 1}]->(b)",userA=userA,userB=userB)
def create_and_return_greeting(tx, message):
    result = tx.run("CREATE (a:Greeting) "
                        "SET a.message = $message "
                        "RETURN a.message + ', from node ' + id(a)", message=message)
    return result.single()[0]

In [5]:
teams = pd.read_csv(filepathhelper.path(dataset,'team.csv'))
closeresolve = pd.read_csv(filepathhelper.path(dataset,'closeresolve.csv'),sep=';')
teams = teams[(teams['issuekey'].isin(closeresolve['issuekey']))]
assignees = pd.read_csv(filepathhelper.path(dataset,'assignee.csv'),sep=';')
assignees = assignees[(assignees['issuekey'].isin(teams['issuekey']))]
assignees.set_index('issuekey',inplace=True)

In [6]:
trainissue = pd.read_csv(filepathhelper.path(dataset,'trainissuekey.csv'))
teams = teams[(teams['issuekey'].isin(trainissue['issuekey']))]

# Create All username

In [7]:
allpp = set(teams['dev'].unique())
allpp.update(teams['tester'].unique())
allpp.update(teams['peer'].unique())
allpp.update(teams['integrator'].unique())
allpp.update(assignees['username'].unique())
len(allpp)

688

In [8]:
with driver.session() as session:
#     session.read_transaction(print_friends_of, "Alice")
    for i in tqdm(allpp):
        session.write_transaction(create_person, i)


/home/noppadolass/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


# Create Relationship

In [9]:
teamdict = {}
for key,issue in tqdm(teams.iterrows(),total=teams.shape[0]):
    issuekey = issue['issuekey']
    team = set()
    if str(issue['dev'])!='nan':
        team.add(issue['dev'])
    if str(issue['integrator'])!='nan':
        team.add(issue['integrator'])
    if str(issue['peer'])!='nan':
        team.add(issue['peer'])
    if str(issue['tester'])!='nan':
        team.add(issue['tester'])
    try:
        asn = assignees.loc[issuekey]['username']
        if str(asn)!='nan':
            team.add(nam)
    except:
        pass
    if issuekey in teamdict:
        teamdict[issuekey].update(team)
    else:
        teamdict[issuekey]=team
    

/home/noppadolass/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [10]:
with driver.session() as session:
    for t in tqdm(teamdict):
        team = teamdict[t]
        if len(team)>1:
            teamlis = list(team)
            for i in range(len(teamlis)):
                for j in range(i+1,len(teamlis)):
                    p1 = teamlis[i]
                    p2 = teamlis[j]
                    session.write_transaction(create_work_relation, p1,p2)
                    session.write_transaction(create_work_relation, p2,p1)

/home/noppadolass/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  
